<a href="https://www.kaggle.com/code/nielco/calorie-predict?scriptVersionId=238254236" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")
submission = pd.read_csv("/kaggle/input/playground-series-s5e5/sample_submission.csv")

In [3]:
train.columns

Index(['id', 'Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate',
       'Body_Temp', 'Calories'],
      dtype='object')

In [4]:
cols = ['Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate',
       'Body_Temp']
X_train = train[cols]
y_train = train["Calories"]
X_test = test[cols]

In [5]:
X_train['Height_m'] = X_train['Height'] / 100
X_train['BMI'] = X_train['Weight'] / (X_train['Height_m'] ** 2)
X_train['Age_Group'] = X_train['Age'].apply(lambda x: 'young' if x < 30 else 'middle' if x < 60 else 'old')
X_train['HR_per_min'] = X_train['Heart_Rate'] / X_train['Duration']
X_train['Body_Load'] = X_train['Weight'] * X_train['Duration'] * X_train['Heart_Rate'] / 10000
X_train['High_Temp'] = (X_train['Body_Temp'] > 37.5).astype(int)
X_train['Is_Male'] = (X_train['Sex'] == 'male').astype(int)

X_test['Height_m'] = X_test['Height'] / 100
X_test['BMI'] = X_test['Weight'] / (X_test['Height_m'] ** 2)
X_test['Age_Group'] = X_test['Age'].apply(lambda x: 'young' if x < 30 else 'middle' if x < 60 else 'old')
X_test['HR_per_min'] = X_test['Heart_Rate'] / X_test['Duration']
X_test['Body_Load'] = X_test['Weight'] * X_test['Duration'] * X_test['Heart_Rate'] / 10000
X_test['High_Temp'] = (X_test['Body_Temp'] > 37.5).astype(int)
X_test['Is_Male'] = (X_test['Sex'] == 'male').astype(int)

/tmp/ipykernel_13/2336146863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Height_m'] = X_train['Height'] / 100


In [6]:
X_train

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Height_m,BMI,Age_Group,HR_per_min,Body_Load,High_Temp,Is_Male
0,male,36,189.0,82.0,26.0,101.0,41.0,1.89,22.955684,middle,3.884615,21.5332,1,1
1,female,64,163.0,60.0,8.0,85.0,39.7,1.63,22.582709,old,10.625000,4.0800,1,0
2,female,51,161.0,64.0,7.0,84.0,39.8,1.61,24.690405,middle,12.000000,3.7632,1,0
3,male,20,192.0,90.0,25.0,105.0,40.7,1.92,24.414062,young,4.200000,23.6250,1,1
4,female,38,166.0,61.0,25.0,102.0,40.6,1.66,22.136740,middle,4.080000,15.5550,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,male,28,193.0,97.0,30.0,114.0,40.9,1.93,26.040968,young,3.800000,33.1740,1,1
749996,female,64,165.0,63.0,18.0,92.0,40.5,1.65,23.140496,old,5.111111,10.4328,1,0
749997,male,60,162.0,67.0,29.0,113.0,40.9,1.62,25.529645,old,3.896552,21.9559,1,1
749998,male,45,182.0,91.0,17.0,102.0,40.3,1.82,27.472527,middle,6.000000,15.7794,1,1


In [7]:
from xgboost import XGBRegressor

model = XGBRegressor(
    objective='reg:squarederror',  # 기본 회귀
    max_depth=5,
    learning_rate=0.1,
    n_estimators=300,  # 더 복잡한 패턴 학습 가능
    subsample=0.8,     # 과적합 방지
    colsample_bytree=0.8,
    random_state=42
)

In [8]:
cat_cols = X_train.select_dtypes(include='object').columns.tolist()

X = pd.get_dummies(X_train, columns=cat_cols)
X_test = pd.get_dummies(X_test, columns=cat_cols)

In [9]:
model.fit(X, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [10]:
prediction = model.predict(X_test)

In [11]:
prediction = prediction.clip(min=0)

In [12]:
(prediction<0).sum()

0

In [13]:
submission["Calories"] = prediction
submission.to_csv("submission.csv", index = False)

In [14]:
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42,
    n_jobs=-1
)

In [15]:
model2.fit(X, y_train)

RandomForestRegressor(max_depth=8, min_samples_leaf=4, min_samples_split=10,
                      n_jobs=-1, random_state=42)

In [16]:
prediction2 = model2.predict(X_test)

In [17]:
submission["Calories"] = prediction2
submission.to_csv("submission.csv", index = False)

In [18]:
print(prediction)
print(prediction2)

[ 27.53198  106.59558   86.26448  ...  73.297585 165.47084   77.15838 ]
[ 25.41219822 101.90778398  89.3484288  ...  65.80975261 170.06465936
  81.1329096 ]
